In [1]:
#Code Portfolio by Hadi Abdelmawla Containing all useful functions to help future students 

In [ ]:
#code to help with histograms

In [ ]:
#Function in order to read a mass amount of image files and to convert them to RGB
#in order to use this the user must define their image path 
#it returns all the images from a directory into an image dictionary that they can use later 
def read_image_files(image_path, conversion=cv2.COLOR_BGR2RGB):
    image_files = glob.glob(os.path.join(image_path, '*')) #function in order to convert the images pass through the image path and conversion
    image_list = [(os.path.basename(f),cv2.imread(f,conversion)) for f in image_files]
    image_dict = {file:image for (file,image) in image_list}
    return image_dict

In [ ]:
#function to incriment the number of bins when it comes to histogram all the user needs to define is the number of incriments that they want and it returns the number of bins
#this function is meant to be used in a for loop
def bin_number(incriments):
    number_bins = number_of_bins * incriments
    return number_bins

In [ ]:
#function to create histograms from a dictionary of images with 8 bins
#in order to use this function the user must take the image dict given from the function above and pass it into this function
#this function returns all the histograms in a dictionary with the filename as the key
#adjustable parameters are the number of bins and image size from [0,256]
def generate_histogram(image_dict, number_bins=8):
    histogram_dict = dict()
    for filename in image_dict: # function to take the images and create histograms for each one 
        image = image_dict[filename]
        hist0 = cv2.calcHist([image], [0], None, [number_bins], [0,256])
        hist1 = cv2.calcHist([image], [1], None, [number_bins], [0,256])
        hist2 = cv2.calcHist([image], [2], None, [number_bins], [0,256])
        overall_hist = np.concatenate([hist0,hist1,hist2]).ravel()
        hist = overall_hist / overall_hist.sum()
        histogram_dict[filename] = hist
    return histogram_dict

In [ ]:
#Function to compare histogram values
#in order to use this function the user will have to generate their histograms for seperate classes in this case it was 
#class and target but the user can change that to their likeing they also have to pass in the image names
#another thing that must be done to use this is create the OPENCV_METHODS if you don't know how to do that just google it
#the return of this function is a dictionary containg the comparision results
def compare_histogram(image_name, target_hist_base, class_histogram_dict):
    results_dict = dict()
    for (methodName, method, reverse) in OPENCV_METHODS:
        results = {k : cv2.compareHist(target_hist_base, hist, method) for (k,hist) in sorted(class_histogram_dict.items())} #compare the histograms pass in the targets and class
        results = sorted([(v, k) for (k, v) in results.items()], reverse = reverse)
        results_dict[methodName] = results
        return results_dict

In [ ]:
#function in order to compare histogram values between the target and myster images 
#in order to use this the user will have to pass through the target image name mystery image name the target histograms and mystery histograms also know as target_hist_base_two
#this function returns the compared mystery results in a dictionary after they have been compared
def compare_mystery(image_name, mystery_image_name, target_hist_base, target_hist_base_two):
    mystery_results_dict = dict()
    for (methodName, method, reverse) in OPENCV_METHODS:
        mystery_results = {image_name : cv2.compareHist(target_hist_base, target_hist_base_two, method)}

        mystery_results_dict[methodName] = mystery_results #create a nested dictionary containing the results of the mystery images compared with the targets
    return mystery_results_dict

In [ ]:
#This function calculates the scores of a comparision test for histograms
#in order to use this function the user will have to pass in a tuple that contains the score and filename
#from there the code will computate the avg median max min if the avg is less than median and a dictionary of all the average scores
def calculate_score_range(histogram_scores_tuple): #calculate the range of scores 
    preferredTest = dict()
    scores = [score for (score,filename) in histogram_scores_tuple] #grab the file scores from the target class compare results
    median_score = scores[int(len(scores)/2)]
    max_score = max(scores)
    min_score = min(scores)
    avg_score = sum(scores)/len(scores) #calculate the median avg max and min
    if(avg_score < median_score): 
        pTest= [methodName]
        preferredTest[methodName]= (avg_score)

    avg_score_dict[methodName] = avg_score

    return (avg_score, median_score, max_score, min_score, preferredTest, avg_score_dict)

In [ ]:
#code to help with face detection

In [ ]:
#Function to process images from a directory
#have to pass the face cascade which if you don't know what it is google it
#the directory where the files are stored 
# the function will convert the image into gray scale rgb and detect the faces in the file and return it all in one image dictionary
def process_images_from_directory(faceCascade, directory, scaleFactor=1.1, minNeighbors=8): # process the images from the directory and create faces for them
    imageDictionary = dict() 
    for directoryPath, directoryNames, fileNames in os.walk(directory):
        for fileName in fileNames:
            imageFile = os.path.join(directoryPath, fileName) 
            # Reads image into RGB for histogram comparison
            img = cv2.imread(imageFile)
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            faces = faceCascade.detectMultiScale(img_gray, scaleFactor=scaleFactor, minNeighbors=minNeighbors)
            print(faces)
            print(imageFile)
            imageDictionary[fileName] = (img_gray,img_rgb, faces)  
    return imageDictionary

In [ ]:
#function to take an image and detect all the skin in it
#then it goes further to mask the image in order that it only shows the skin of darker people
#you can adjust this code by changing the filter of 205,255,255 to change the color of skin you want masked 
#the inputs are just an image 
#the output is a directory of the masked image normally then in rgb
def skinColor_mask_hist (da_img): # this where i take the image passed and mask it for only brown skin colors
    skin_mask_dict = dict()
    rgb_mask_dict = dict()

    imageYCrCb = cv2.cvtColor(da_img,cv2.COLOR_BGR2YCR_CB)
    skinRegionYCrCb = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb) # I detect the region in which there is skin
    daimg_rgb = cv2.cvtColor(da_img, cv2.COLOR_BGR2RGB)
    skinYCrCb = cv2.bitwise_and(daimg_rgb, daimg_rgb, mask = skinRegionYCrCb) # then mask the original image to only show skin
    skin_mask_dict = skinYCrCb
    lower = np.array([190, 0, 0], dtype = "uint8")
    upper = np.array([205,255,255], dtype = "uint8") # I then define the limits of brown that I want in RGB form
    mask = cv2.inRange(skinYCrCb, lower, upper)
    output = cv2.bitwise_and(skinYCrCb, skinYCrCb, mask = mask) # i then mask the masked skin image to only show pixels that satisfy the limits of brown
    rgb_mask_dict = output
    return skin_mask_dict, rgb_mask_dict

In [ ]:
#two functions to compare the amount of skin in each image 
#you have to pass in histograms and just the base image you want to compare against where it says obamatemplate0.jpg
#Very useful in trying to limit a folder of images to only darker skin
#returns the scores so you can compare
def compare_histogram(image_name, target_hist_base, class_histogram_dict): # same function from pokemon
    results_dict = dict()
    for (methodName, method, reverse) in OPENCV_METHODS:
        results = {k : cv2.compareHist(target_hist_base, hist, method) for (k,hist) in sorted(class_histogram_dict.items())} #compare the histograms pass in the targets and class
        results = sorted([(v, k) for (k, v) in results.items()], reverse = reverse)
        results_dict[methodName] = results
        
    return results_dict
def compare_hist_template(masked_hist): # here is where I compare the templates against each other
    template_scores = dict()
    for filename in masked_hist:
        if(filename == 'ObamaTemplate0.jpg'):
            continue
        template_scores[filename] = (cv2.compareHist(masked_hist_obama['ObamaTemplate0.jpg'], masked_hist[filename], cv2.HISTCMP_INTERSECT))
    return template_scores

In [ ]:
#code to help with creating a cnn

In [ ]:
#code to define a model for a cnn
#the user can change the amount of layers and size of each one
#make sure you put in the correct inputshape when it asks here it is currently 28,28,1
#also make sure to have correct number of folders you want to test where it say dense at the bottom for this there
#were 10 folders to train thus the 10
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform',input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    
    # compile model
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [ ]:
#Function to create a small VGG model the user will have to pass in how many layers they want from one to five which is all of them
# 5 layers the user needs to make sure to update the input shape and number of folders that are being trained 
#returns the test model which is ready to be trained with epochs
def smallVgg(howmany): #created a smallvgg with the option to limit how many times it is added
    test_model = Sequential()
    test_model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), padding='same', activation='relu'))
    test_model.add(MaxPooling2D((3,3), strides=1))
    if (howmany == 'first'):
        test_model.add(Dense(256, activation='relu'))
        test_model.add(Dense(10, activation='softmax'))
        test_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        return test_model
    test_model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    test_model.add(MaxPooling2D((3,3), strides=1))
    if howmany == 'second':
        test_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        return test_model
    test_model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    test_model.add(MaxPooling2D((3,3), strides=1))
    if howmany == 'third':
        test_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        return test_model
    test_model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    test_model.add(MaxPooling2D((3,3), strides=1))
    if (howmany == 'fourth'):
        test_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        return test_model
    test_model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    test_model.add(MaxPooling2D((3,3), strides=1))
    test_model.add(Flatten())
    test_model.add(Dense(256, activation='relu'))
    test_model.add(Dense(10, activation='softmax'))
    test_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return test_model

In [ ]:
#code to help with image detections

In [ ]:
#this code is very heavily commented in order for the user to understand 
#rewrote the above into a function as asked for in the hw above part A
def reimage (rects, image, defaultSize):
    # initialize the list of region proposals that we'll be classifying
    # along with their associated bounding boxes
    proposals = []
    boxes = []

# loop over the region proposal bounding box coordinates generated by
# running selective search
    for (x, y, w, h) in rects:
	# if the width or height of the region is less than 10% of the
	# image width or height, ignore it (i.e., filter out small
	# objects that are likely false-positives)
        if w / float(W) < 0.1 or h / float(H) < 0.1:
            continue

	# extract the region from the input image, convert it from BGR to
	# RGB channel ordering, and then resize it to 224x224 (the input
	# dimensions required by our pre-trained CNN)
        roi = image[y:y + h, x:x + w]
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        roi = cv2.resize(roi, defaultSize)

	# further preprocess by the ROI
        roi = img_to_array(roi)
        roi = preprocess_input(roi)

	# update our proposals and bounding boxes lists
        proposals.append(roi)
        boxes.append((x, y, w, h))
    
# convert the proposals list into NumPy array and show its dimensions
    proposals = np.array(proposals)
    print("[INFO] proposal shape: {}".format(proposals.shape))
    return proposals, boxes

In [ ]:
#This code is also heavily commented to make it easy for the user to comprehend what is going on
#rewrote the function above in order to satisfy part A
def winapproach (strides, image, fixedwin): #strides has to be in the form
    #of (y,x) with x being the stride length in the x direction and y being 
    #the stride length in the y direction
    #fixed win is the fixed size windows
    # initialize the list of region proposals that we'll be classifying
    # along with their associated bounding boxes
    proposals = []
    boxes = []
    newx = 0
    newy = 0
    xlim = image.shape[1] - fixedwin[1] #to find the limit in the x direction
    #subtract the shaped of the image by the  direction of the fixedwin
    ylim = image.shape[0] - fixedwin[0]
    xrange = int (xlim / strides) #then divide it by the stride in order to know
    #how many times the fixed win fits in the image given the stride length
    yrange = int(ylim / strides)
# loop over the region proposal bounding box coordinates generated by
# running selective search
    for x in range (0, xrange):
        newx = newx + strides
        newy=0
        for y in range (0, yrange):
            
            roi = image[newy:fixedwin[0] + newy, newx:fixedwin[1] + newx]
#Now the roi has starting coordinates of 0,0 for the first iteration then it 
#moves by whatever the strid length is for each iteration the corresponding 
#box is the whatever the staring coordinate plus the fixed window size
            #print(newx,newy)
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            roi = cv2.resize(roi, (224,224))
            
            newy = newy + strides
            roi = img_to_array(roi)
            roi = preprocess_input(roi)
            proposals.append(roi)
            boxes.append((newx, newy, fixedwin[1], fixedwin[0]))
    proposals = np.array(proposals)
    print("[INFO] proposal shape: {}".format(proposals.shape))
    return proposals, boxes

In [ ]:
#function to get the time that a classification takes 
# classify each of the proposal ROIs using ResNet and then decode the
# predictions
#created this into a function to return predictions and time it takes for part F
def classifiying(someproposals):
    print("[INFO] classifying proposals...")
    startTime = cv2.getTickCount() # get the time it takes to do model.predict
    preds = model.predict(someproposals)
    endTime = cv2.getTickCount()
    timePredict = (e2 - e1)/ cv2.getTickFrequency()
    preds = imagenet_utils.decode_predictions(preds, top=1)
    return preds, timePredict

In [ ]:
#function in order to get the labels and confidence of an image detection have to pass in predictions the boxes the label filters and confidence
#created the above function into a code with a conf variable for part d
def findlabels(preds, boxes, labelFilters, conf):
# initialize a dictionary which maps class labels (keys) to any
# bounding box associated with that label (values)
    labels = {}

# loop over the predictions
    for (i, p) in enumerate(preds):
	# grab the prediction information for the current region proposal
        (imagenetID, label, prob) = p[0]
        print (label)

	# only if the label filters are not empty *and* the label does not
	# exist in the list, then ignore it
        if labelFilters is not None and label not in labelFilters:
            continue

	# filter out weak detections by ensuring the predicted probability
	# is greater than the minimum probability
        if prob >= conf:
		# grab the bounding box associated with the prediction and
		# convert the coordinates
            print(i)
            if(i==540):
                return labels, box
            if(boxs[i]>((H,W))):
                continue
            (x, y, w, h) = boxs[i]

            box = (x, y, x + w, y + h)

		# grab the list of predictions for the label and add the
		# bounding box + probability to the list
            L = labels.get(label, [])
            L.append((box, prob))
            labels[label] = L
    return labels, box

In [ ]:
#Code in order to find a desiered object in an image with the predictions 
# all the user has to pass in the labels and the code does the rest and returns the prediction where the desired object
#is in an image with a box surronding it 
def finalpred(labels):
# loop over the labels for each of detected objects in the image
    for label in labels.keys():
	# clone the original image so that we can draw on it
        print("[INFO] showing results for '{}'".format(label))

	# loop over all bounding boxes for the current label
        for (box, prob) in labels[label]:
		# draw the bounding box on the image
            (startX, startY, endX, endY) = box
            cv2.rectangle(proposals_image, (startX, startY), (endX, endY),
                (0, 255, 0), 2)


	# extract the bounding boxes and associated prediction
	# probabilities, then apply non-maxima suppression
        boxes = np.array([p[0] for p in labels[label]])
        proba = np.array([p[1] for p in labels[label]])
        boxes = non_max_suppression(boxes, proba)

	# loop over all bounding boxes that were kept after applying
	# non-maxima suppression
        for (startX, startY, endX, endY) in boxes:
		# draw the bounding box and label on the image
            cv2.rectangle(annotated_image, (startX, startY), (endX, endY),
                (0, 255, 0), 2)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.putText(annotated_image, label, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 1)
    return proposals_image, boxes

In [ ]:
#functions for block search below

In [ ]:
#Code to do a full search on an np.array 
#the user must pass in the xrange yrange which is a set of cordinates so it must be a list the image table which is a np.array
#and they must pass in the square size that they want to use to search
#this function returns the area where it is a its max in the array and the position of the max
def fullsearch(xrange,yrange,w,imageTable, squaresize):
    w = np.zeros((15,15))
    x=0
    z=0
    for (i) in range (xrange[0],xrange[1]):
        #print(i)
        for j in range (yrange[0],yrange[1]):
         #   print (j)
        #print(i)
            w[i][j] = imageTable[i][j]
            z=z+1
            if (z==squaresize):
                z=0
        x=x+1
    maxvalue = np.amax(w)
    index = np.where(w == np.amax(w))
    return maxvalue, index

In [ ]:
#function to complete a three s search on an np.array
#the user must pass in the range for x and y in coordinate form the squaresize they want the test array they want to pass over the np.array
#and the np array in question 
#this function returns the max value determined and the position of it
def threess(xrange,yrange,imageTable,squaresize,testarray):
    w = np.zeros((15,15))
    x=0
    z=0
    for (i) in range (xrange[0],xrange[1]):
        #print(i)
        for j in range (yrange[0],yrange[1]):
         #   print (j)
        #print(i)
            if(i>(squaresize)):
                x = i-7
            if(j>(squaresize)):
                z = j-7
            if(testarray[x][z]==0):
                continue
            w[i][j] = imageTable[i][j]
            
            #z=z+1
            #if (z==squaresize):
             #   z=0
        #x=x+1
    maxvalue = np.amax(w)
    index = np.where(w == np.amax(w))
    return maxvalue, index

In [ ]:
#functions to help with creating a cnn
#pass in two paths containing your test and train images and the function will rescale them and use keras to 
#create and image generator of them and return its data
def upload_image(train_path, test_path):
    #function to upload image from a path 
    train_d = ImageDataGenerator(rescale = 1. /255, preprocessing_function=preprocess_input)
    test_d = ImageDataGenerator(rescale = 1. /255, preprocessing_function=preprocess_input)
    #use of imgdatagenerator in order to rescale all the test and train images
    train_g = train_d.flow_from_directory(train_path, target_size=(177, 254), batch_size=32)
    print(train_g.class_indices)
    #use flow from directory to store them and indicies and have them ready to be processed
    test_g = test_d.flow_from_directory(test_path, target_size=(177, 254), batch_size=32)
    #the target size is important to have but shouldn't matter too much because all the images are
    #scaled to the proper size earlier
    print(test_g.class_indices)
    return (train_d, test_d, train_g, test_g)


In [ ]:
#code if you ever want to display a random image from an image path all you have to pass is the the pass and train generator from the function above
#then it shows a random image
def random_image(train_path, train_generator):
    #use this function to show a random image contained inside the test generator
    rand_img = np.random.choice(train_generator.filenames)
    #using numpy to make a random selection inside the train_generator filenames
    img = Image.open(train_path + rand_img)
    #use this to open the image from the path from train generator and random file
    print(train_path+random_img)
    print(img.size)
    plt.imshow(img)
    return

In [ ]:
#function to predict the class of an image 
#this can be used for more than just card types all depends on the labels the user used and it can be adjusted accordingly
#it also returns the prediction of the class type
def get_class(path): #use this to predict the class of a random image
    the_img = Image.open(path).convert('RGB')
    imgn = the_img.resize((254,177))
    plt.imshow(the_img)
    plt.axis('off')
    x = np.asarray(imgn, np.float32)[None]
    x = preprocess_input(x)
    y = mod.predict(x)
    
    preds = label_dict[np.argmax(y)]
    return preds

In [ ]:
#code to help with sorting dictionaries and lists

In [ ]:
#pass in one dictionary and two lists and the code will return to you the name which is from the dictionary the values and index
#returns it in list form
def results_in_tuple(Results, classinfo, resultobjects): # solution for part 1
    newlist = []
    numbersSorted = {k:v for (k,v) in enumerate(Results)} #create a sorted list here with the scores by with the index values on them in order to not lose the position of the value when sorting
    newsort = sorted(numbersSorted.values(),reverse=False)
    for x,y in enumerate(newsort): #go through the sorted list from above
        new_dict = {x: (k) for (v,k) in (classinfo[resultobjects[x]].items())} #store the names of each object in a dict
        newlist.append((y)) #store the position first
        newlist.append((new_dict[x])) #the name second
        newlist.append((x))#the index third which is position of it in the objects and scores list
    return newlist # return the list

In [ ]:
#very useful code to help with creating a dictionary when the user has multiple values for something that would have the same key
#pass in the same things as above
#the output the key value and index in dictionary form 
def results_dictionary(Results, classinfo, resultobjects): #now for task2
    final_dict = defaultdict(list)#use default dict in order to create a dict with duplicate keys
    numbersSorted = {k:v for (k,v) in enumerate(Results)}
    newsort = sorted(numbersSorted.values(),reverse=False)#same process as above for the sort
    for h,i in enumerate(newsort):
        new_dict = {h: (k) for (v,k) in (classinfo[resultobjects[h]].items())} #h is index i is score       
        temp = (i,h)#create a temp of the values in order to use append
        final_dict[new_dict[h]].append(temp)#append the score and index in the dictionary to support multiple keys
    return (final_dict)